<b>Name</b> : Sindhura Bagodu Ramachandra

<b>USC ID</b> : 4628596426

<b>Github username</b> : SindhuraBagodu

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import warnings
import random
import os
import shutil
import math
from PIL import Image
from sklearn.preprocessing import LabelEncoder
# import tensorflow as tf
# from keras.preprocessing.image import ImageDataGenerator

##### 1. Transfer Learning for Image Classification1

It is highly recommended that you complete this project using Keras2 and Python.

(a) In this problem, we are trying to build a classifier that distinguishes images of 20 bird species. You are provided with text data in twenty folders.

###### (b) Data Exploration and Pre-processing

i. Images in each class are given in separate folders. The file Classes.xlsx provides the classes assigned to the bird species images in each folder. Therefore, you encode your classes using one-hot encoding and Classes.xlsx.

ii. Randomly select d0.7ni images from each folder as your training set, [0.15ni] as validation set, and the rest as your test set, where ni is the number of images in folder i and dxe is the ceiling of x. 


In [12]:
try:
    os.makedirs('./output')
    os.makedirs('./output/train')
    os.makedirs('./output/test')

    os.listdir('./output')
    
except OSError as e:
    print(e)

In [13]:
root_dir = '../Data/images'
classes = ['005.Crested_Auklet', '013.Bobolink', '015.Lazuli_Bunting', '023.Brandt_Cormorant', '040.Olive_sided_Flycatcher', '041.Scissor_tailed_Flycatcher', '067.Anna_Hummingbird', '072.Pomarine_Jaeger', '076.Dark_eyed_Junco', '081.Pied_Kingfisher', '082.Ringed_Kingfisher', '086.Pacific_Loon', '099.Ovenbird', '104.American_Pipit', '127.Savannah_Sparrow', '135.Bank_Swallow', '141.Artic_Tern', '149.Brown_Thrasher', '156.White_eyed_Vireo', '168.Kentucky_Warbler']

for clss in classes:
#     print('------------' + clss + '-------------')
    dirtry = root_dir + '/' + clss
    files = os.listdir(dirtry)
    np.random.shuffle(files)
    
    base_outdir = './output/'

    for folder in ['train', 'test', 'validation']:
        target_dir = base_outdir + folder
        os.makedirs(target_dir + '/' + clss)
        target_class = target_dir + '/' + clss

        if folder == 'train':
            print(math.ceil(0.7*len(files)))
            images_to_pass = files[: math.ceil(0.7*len(files))]
            for img in images_to_pass:
                img = dirtry + '/' + img
                shutil.copy(img, target_class)
                
        elif folder == 'validation':
            print(math.ceil(0.85*len(files)))
            images_to_pass = files[math.ceil(0.7*len(files)): math.ceil(0.85*len(files))]
            for img in images_to_pass:
                img = dirtry + '/' + img
                try:
                    shutil.copy(img, target_class)
                except OSError as e:
                      print(e)
        else:
            images_to_pass = files[math.ceil(0.85*len(files)):]
            for img in images_to_pass:
                img = dirtry + '/' + img
                shutil.copy(img, target_class)


31
38
42
51
41
50
42
51
42
51
42
51
42
51
42
51
42
51
42
51
42
51
42
51
42
51
42
51
42
51
42
51
41
50
42
51
42
51
42
51



iii. In order for all the images to have the same size, zero-pad or resize the images in your dataset. This can be done using various tools, including OpenCV.

In [5]:
def resize(path):
    
    for clss in classes:
        dirtry = path + '/' + clss
        files = os.listdir(dirtry)
        for item in files:
            f_img = dirtry+"/"+item
            img = Image.open(f_img)
            img = img.resize((224,224))
            img.save(f_img)            

resize('./output/train/')
resize('./output/validation/')
resize('./output/test/')

##### (c) Transfer Learning

i. When dealing with classification of relatively small image datasets, deep networks may not perform very well because of not having enough data to train them. In such cases, one usually uses transfer learning, which uses deep learning models that are trained on very large datasets such as ImageNet as feature extractors. The idea is that such deep networks have learned to extract meaningful features from an image using their layers, and those features can be used in learning other tasks. In order to do that, usually the last layer or the last few layers of the pre-trained network are removed, and the response of the layer before the removed layers to the images in the new dataset is used as a feature vector to train one more multiple replacement layers. The dataset in this task has only around 50-60 images per class. Given that we have 20 classes, training a deep network with such a small dataset may not yield desirable results. In this project, you will use pre-trained models EfficientNetB0 and VGG16. For both pre-trained networks, you will only train the last fully connected layer, and will freeze all layers before them (i.e. we do not change their parameters during training) and use the outputs of the penultimate layer in the original pre-trained model as the features extracted from each image.

In [15]:
labelencoder = LabelEncoder()
bird_df = pd.DataFrame(classes, columns=['Bird_class'])
bird_df['Birds_encoded'] = labelencoder.fit_transform(bird_df['Bird_class'])
bird_df

,Bird_class,Birds_encoded
0,005.Crested_Auklet,0
1,013.Bobolink,1
2,015.Lazuli_Bunting,2
3,023.Brandt_Cormorant,3
4,040.Olive_sided_Flycatcher,4
5,041.Scissor_tailed_Flycatcher,5
6,067.Anna_Hummingbird,6
7,072.Pomarine_Jaeger,7
8,076.Dark_eyed_Junco,8
9,081.Pied_Kingfisher,9


In [17]:
classes_label = ['005.Crested_Auklet', '013.Bobolink', '015.Lazuli_Bunting', '023.Brandt_Cormorant', '040.Olive_sided_Flycatcher', '041.Scissor_tailed_Flycatcher', '067.Anna_Hummingbird', '072.Pomarine_Jaeger', '076.Dark_eyed_Junco', '081.Pied_Kingfisher', '082.Ringed_Kingfisher', '086.Pacific_Loon', '099.Ovenbird', '104.American_Pipit', '127.Savannah_Sparrow', '135.Bank_Swallow', '141.Artic_Tern', '149.Brown_Thrasher', '156.White_eyed_Vireo', '168.Kentucky_Warbler']

train_datagen = ImageDataGenerator(rotation_range=20,	zoom_range=0.15, rescale=1./255)
train_generator = train_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/output/train', target_size=(224,224), batch_size=5,classes=classes_label, class_mode='categorical', shuffle=True)

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/output/validation',target_size=(224,224),batch_size=5,classes=classes_label,class_mode='categorical',shuffle=False)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/output/test',target_size=(224,224),batch_size=5,classes=classes_label,class_mode='categorical',shuffle=False)


NameError: name 'ImageDataGenerator' is not defined

In [26]:
!pip install tensorflow

You should consider upgrading via the 'C:\Users\SINDHURA\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip3 install --upgrade tensorflow

  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.5.1
    Uninstalling google-auth-oauthlib-0.5.1:
      Successfully uninstalled google-auth-oauthlib-0.5.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.12.2
    Uninstalling tensorboard-1.12.2:
      Successfully uninstalled tensorboard-1.12.2
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 1.12.0
    Uninstalling tensorflow-1.12.0:
      Successfully uninstalled tensorflow-1.12.0


You should consider upgrading via the 'C:\Users\SINDHURA\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
import tensorflow as tf

In [7]:
tf.version.VERSION

AttributeError: module 'tensorflow' has no attribute 'version'